In [7]:
## OFFICAL_:

import tkinter as tk
from tkinter import messagebox, scrolledtext
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
import time
import warnings
import gc

# Suppress warnings
warnings.filterwarnings('ignore')

# Load and preprocess data
dt = 'V:/Departamentos/DVM/Transformação Digital/Projetos/064 - Otimização de Processo com IA no VOD (Ciência de Dados)/VOD_JUL_2024.xlsx'
dt = pd.read_excel(dt)
dt = pd.DataFrame(dt)
dt['Tempo de depressão de vácuo(min)'].replace(',', '.', inplace=True)
dt['Tempo de depressão de vácuo(min)'] = pd.to_numeric(dt['Tempo de depressão de vácuo(min)'], errors='coerce')

dt['Teor de C após a descarburação(%)'].replace(',', '.', inplace=True)
dt['Teor de C após a descarburação(%)'] = pd.to_numeric(dt['Teor de C após a descarburação(%)'], errors='coerce')
dt.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2', 'Agitação média do TruStir durante a descarburação'], inplace=True)
dt.replace('Boa', 1, inplace=True)
dt.replace('Ruim', 0, inplace=True)
dt = pd.get_dummies(dt)
for i in dt.columns:
    median_value = dt[i].median()
    dt[i].fillna(median_value, inplace=True)
dt = dt.replace({False: 0, True: 1})

# Define optimization
dt['opt'] = 0
dt.loc[
    (dt['Teor de C após a descarburação(%)'] < 0.009) &
    (dt['Temperatura do aço após a descarburação(°C)'] >= 1680) &
    (dt['Temperatura do aço após a descarburação(°C)'] <= 1700),
    'opt'
] = 1

# Initialize global variables
columns_ = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]

stp = [[0, 0, 0, 0, 0]]
contr = pd.DataFrame(stp, columns=columns_)

ent = dt[['Teor de C antes do VOD(%)',
          'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
          'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
          'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
          'Vida de plug da panela', 'Vida da panela']]
ent_ = ent.sample(n=1)

# Update stp function
def update_stp():
    global stp, contr
    try:
        stp_values = [float(entry.get()) for entry in entries]
        stp = [stp_values]
        contr = pd.DataFrame(stp, columns=columns_)
        messagebox.showinfo("Success", "Values updated successfully!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers.")

# Run processing loop function
def run_loop():
    global contr, dt, ent_

    # Initialize x_ and y_ to be used later
    x_ = dt[[col for col in columns_ if col in dt.columns]]
    y_ = dt['opt']

    # Initialize loop variables
    condition_met = False
    start_time = time.time()
    time_limit = 40
    iteration = 0

    # Precompute constraints if needed
    constraints = {
        'Tempo abaixo de 10 mbar(min)': (9.65, 23),
        'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.52, 6.83),
        'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.32),
        'Volume de O2 soprado(m3)': (115.39, 248.45),
        'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
    }

    while not condition_met:
        try:
            # Track elapsed time
            elapsed_time = time.time() - start_time
            if elapsed_time > time_limit:
                result_text.insert(tk.END, "Time limit exceeded. Stopping the loop.\n")
                break

            iteration += 1

            # Reset index for contr DataFrame
            contr.reset_index(drop=True, inplace=True)

            # Concatenate and process DataFrames
            tt2 = pd.concat([ent_, contr], axis=1)
            tt2.fillna(0, inplace=True)
            tt2 = pd.DataFrame(tt2.sum()).T

            # Apply constraints
            for col, (a_min, a_max) in constraints.items():
                if col in tt2.columns:
                    tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

            tt2_test = tt2[columns_]

            # Train-test split
            x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.5, random_state=42)

            # Train the Logistic Regression model
            model = LogisticRegression(penalty='l2', solver='saga', class_weight='balanced', C=0.1)
            model.fit(x_train, y_train)
            y_pred2 = model.predict(tt2_test)
            y_pred_prob2 = model.predict_proba(tt2_test)[:, 1]  # Probability of positive class
            y_pred_score2 = model.score(x_test, y_test)

            # Fit the Logit model from Statsmodels
            logit_model = sm.Logit(y_train, x_train)
            results = logit_model.fit(method='bfgs', cov_type='HC3')

            # Extract coefficients and compute exponentiated coefficients
            co = results.params
            exp_coeff = np.exp(co) / (np.exp(co) + 1)
            exp_coeff_df = pd.DataFrame({'exp_coeff': exp_coeff})

            # Extract results summary
            results_summary = results.summary()
            results_as_html = results_summary.tables[1].as_html()
            df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
            df_results['exp_coeff'] = exp_coeff_df

            # Filter significant variables
            impact_var2 = df_results[df_results['P>|z|'] < 0.05]

            # Calculate results
            results_list = []
            for i in impact_var2.index:
                value = impact_var2.loc[i, 'exp_coeff']
                if i in tt2_test.columns:
                    correction = tt2_test[i].median()
                    result = (value - 0.5) * correction if value != 0.5 else 0
                    results_list.append({'Column': i, 'Alteração': result})

            results_df_ = pd.DataFrame(results_list).T
            results_df_.columns = results_df_.iloc[0]
            results_df_ = results_df_[1:]

            # Common columns
            common_columns = list(set(contr.columns) & set(results_df_.columns))
            contr_common = contr[common_columns]
            results_common = results_df_[common_columns]

            # Combine and update
            combined_df = pd.concat([contr_common, results_common], ignore_index=True)
            final = combined_df.sum().reset_index()
            final_df = final.T
            final_df.columns = final_df.iloc[0]
            final_df = final_df[1:]

            # Update contr with final_df values
            contr.update(final_df[common_columns])
            contr.reset_index(drop=True, inplace=True)

            # Garbage collection
            gc.collect()

            # Check condition
            if np.all(y_pred_prob2 >= 0.99) and np.all(y_pred2 == 1):
                condition_met = True

        except Exception as e:
            continue

    # Display results
    result_text.insert(tk.END, "Processing complete.\n")
    
    # Display only the setup columns from tt2
    result_text.insert(tk.END, "\nUpdated setup columns values:\n")
    for col in columns_:
        if col in tt2.columns:
            result_text.insert(tk.END, f"{col}: {tt2[col].values[0]:.4f}\n")

# Set up the Tkinter window
root = tk.Tk()
root.title("Processing Loop")

# Set up the input section for `stp`
tk.Label(root, text="Enter values for stp:").pack()
entries = []
for col in columns_:
    tk.Label(root, text=col).pack()
    entry = tk.Entry(root)
    entry.pack()
    entries.append(entry)

update_button = tk.Button(root, text="Update stp", command=update_stp)
update_button.pack()

run_button = tk.Button(root, text="Run Processing Loop", command=run_loop)
run_button.pack()

result_text = scrolledtext.ScrolledText(root, width=100, height=30)
result_text.pack()

root.mainloop()


Optimization terminated successfully.
         Current function value: 0.531814
         Iterations: 16
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.531814
         Iterations: 16
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.531814
         Iterations: 16
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.531814
         Iterations: 16
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.531814
         Iterations: 16
         Function evaluations: 23
         Gradient evaluations: 23
Optimization terminated successfully.
         Current function value: 0.531814
         Iterations: 16
         Function evaluations: 23
  

KeyboardInterrupt: 

In [11]:
## OFFICIAL com a HUBBER LOSS

import tkinter as tk
from tkinter import messagebox, scrolledtext
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import statsmodels.api as sm
import time
import warnings
import gc

# Suppress warnings
warnings.filterwarnings('ignore')

# Load and preprocess data
dt = 'V:/Departamentos/DVM/Transformação Digital/Projetos/064 - Otimização de Processo com IA no VOD (Ciência de Dados)/VOD_JUL_2024.xlsx'
dt = pd.read_excel(dt)
dt = pd.DataFrame(dt)
dt['Tempo de depressão de vácuo(min)'].replace(',', '.', inplace=True)
dt['Tempo de depressão de vácuo(min)'] = pd.to_numeric(dt['Tempo de depressão de vácuo(min)'], errors='coerce')

dt['Teor de C após a descarburação(%)'].replace(',', '.', inplace=True)
dt['Teor de C após a descarburação(%)'] = pd.to_numeric(dt['Teor de C após a descarburação(%)'], errors='coerce')
dt.drop(columns=['DATA', 'Agitação média do TruStir durante o sopro de O2', 'Agitação média do TruStir durante a descarburação'], inplace=True)
dt.replace('Boa', 1, inplace=True)
dt.replace('Ruim', 0, inplace=True)
dt = pd.get_dummies(dt)
for i in dt.columns:
    median_value = dt[i].median()
    dt[i].fillna(median_value, inplace=True)
dt = dt.replace({False: 0, True: 1})

# Define optimization
dt['opt'] = 0
dt.loc[(
    dt['Teor de C após a descarburação(%)'] < 0.009) & 
    (dt['Temperatura do aço após a descarburação(°C)'] >= 1680) & 
    (dt['Temperatura do aço após a descarburação(°C)'] <= 1700),
    'opt'] = 1

# Initialize global variables
columns_ = [
    'Vazão média de argônio durante o sopro de O2(Nm3/h)',
    'Pressão média de vácuo durante o sopro de O2(mbar)',
    'Tempo abaixo de 10 mbar(min)',
    'Vazão média de argônio durante a descarburação(Nm3/h)',
    'Volume de O2 soprado(m3)'
]

stp = [[0, 0, 0, 0, 0]]
contr = pd.DataFrame(stp, columns=columns_)

ent = dt[['Teor de C antes do VOD(%)',
          'Teor de Cr antes do VOD(%)', 'Teor de Si antes do VOD(%)',
          'Teor de Mn antes do VOD(%)', 'Teor de Ni antes do VOD(%)', 'Teor de Mo antes do VOD(%)',
          'Temperatura do aço antes do VOD(°C)',
          'Peso de aço antes do VOD(kg) ', 'Borda livre da panela(mm)',
          'Vida de plug da panela', 'Vida da panela']]
ent_ = ent.sample(n=1)

# Huber loss function
def huber_loss(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    loss = np.where(abs_residuals <= delta,
                    0.5 * residuals**2,
                    delta * (abs_residuals - 0.5 * delta))
    return np.sum(loss) / n

def huber_loss_grad(params, X, y, delta=1.0):
    X = np.asarray(X)
    y = np.asarray(y)
    params = np.asarray(params)
    
    n = X.shape[0]
    y_pred = 1 / (1 + np.exp(-np.dot(X, params)))
    residuals = y - y_pred
    abs_residuals = np.abs(residuals)
    
    grad = -np.dot(X.T, np.where(abs_residuals <= delta,
                                 residuals,
                                 delta * np.sign(residuals))) / n
    return grad

# Update stp function
def update_stp():
    global stp, contr
    try:
        stp_values = [float(entry.get()) for entry in entries]
        stp = [stp_values]
        contr = pd.DataFrame(stp, columns=columns_)
        messagebox.showinfo("Success", "Values updated successfully!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers.")

# Run processing loop function
def run_loop():
    global contr, dt, ent_

    # Initialize x_ and y_ to be used later
    x_ = dt[[col for col in columns_ if col in dt.columns]]
    y_ = dt['opt']

    # Initialize loop variables
    condition_met = False
    start_time = time.time()
    time_limit = 5
    iteration = 0

    # Precompute constraints if needed
    constraints = {
        'Tempo abaixo de 10 mbar(min)': (9.65, 23),
        'Vazão média de argônio durante o sopro de O2(Nm3/h)': (3.52, 6.83),
        'Pressão média de vácuo durante o sopro de O2(mbar)': (400, 693.32),
        'Volume de O2 soprado(m3)': (115.39, 248.45),
        'Vazão média de argônio durante a descarburação(Nm3/h)': (4.55, 7.87)
    }

    while not condition_met:
        try:
            # Track elapsed time
            elapsed_time = time.time() - start_time
            if elapsed_time > time_limit:
                result_text.insert(tk.END, "Time limit exceeded. Stopping the loop.\n")
                break

            iteration += 1

            # Reset index for contr DataFrame
            contr.reset_index(drop=True, inplace=True)

            # Concatenate and process DataFrames
            tt2 = pd.concat([ent_, contr], axis=1)
            tt2.fillna(0, inplace=True)
            tt2 = pd.DataFrame(tt2.sum()).T

            # Apply constraints
            for col, (a_min, a_max) in constraints.items():
                if col in tt2.columns:
                    tt2[col] = np.clip(tt2[col], a_min=a_min, a_max=a_max)

            tt2_test = tt2[columns_]

            # Train-test split
            x_train, x_test, y_train, y_test = train_test_split(x_, y_, test_size=0.5, random_state=42)

            # Optimize using Huber loss
            initial_params = np.zeros(x_train.shape[1])
            result = minimize(fun=huber_loss, x0=initial_params, args=(x_train.values, y_train.values),
                              jac=huber_loss_grad, method='BFGS')

            optimal_params = result.x

            # Predicting on tt2
            y_pred_prob = 1 / (1 + np.exp(-np.dot(tt2_test.values, optimal_params)))
            y_pred_binary = (y_pred_prob >= 0.5).astype(int)

            # Display results
           # result_text.insert(tk.END, f"\nIteration: {iteration}\n")
            #result_text.insert(tk.END, f"Predicted Probability: {y_pred_prob[0]:.4f}\n")
            #result_text.insert(tk.END, f"Predicted Class: {y_pred_binary[0]}\n")

            # Garbage collection
            gc.collect()

            # Check condition
            if np.all(y_pred_prob >= 0.99) and np.all(y_pred_binary == 1):
                condition_met = True

        except Exception as e:
            continue

    # Display results
    result_text.insert(tk.END, "Processing complete.\n")
    
    # Display only the setup columns from tt2
    result_text.insert(tk.END, "\nUpdated setup columns values:\n")
    for col in columns_:
        if col in tt2.columns:
            result_text.insert(tk.END, f"{col}: {tt2[col].values[0]:.4f}\n")

# Set up the Tkinter window
root = tk.Tk()
root.title("Processing Loop")

# Set up the input section for `stp`
tk.Label(root, text="Enter values for stp:").pack()
entries = []
for col in columns_:
    tk.Label(root, text=col).pack()
    entry = tk.Entry(root)
    entry.pack()
    entries.append(entry)

update_button = tk.Button(root, text="Update stp", command=update_stp)
update_button.pack()

run_button = tk.Button(root, text="Run Processing Loop", command=run_loop)
run_button.pack()

result_text = scrolledtext.ScrolledText(root, width=100, height=30)
result_text.pack()

root.mainloop()
